In [1]:
import boto3
import json
import sys
import time

# make sure your aws role has the correct permissions to run textract, see https://docs.aws.amazon.com/textract/latest/dg/api-async-roles.html 
# to configure 

class ProcessType:
    DETECTION = 1
    ANALYSIS = 2    

class DocumentProcessor:
    jobId = ''
    textract = boto3.client('textract')
    sqs = boto3.client('sqs')
    sns = boto3.client('sns')
    roleArn = ''
    bucket = ''
    document = ''
    sqsQueueUrl = ''
    snsTopicArn = ''
    processType = ''


    def __init__(self, role, bucket, document,topic):
        self.roleArn = role  #refers to roleARN, ours = 'arn:aws:iam::760515291717:role/TextractRole'
        self.bucket = bucket
        self.document = document  #refers to name of document being held in bucket
        self.snsTopicArn = topic. #refers to snstopicARN, ours = 'arn:aws:sns:us-west-1:760515291717:nlp_topic'


    def ProcessDocument(self,type):  #refers back to ProcessType class ^^
        jobFound = False
        self.processType=type
        validType=False
        
        #Determine which type of processing to perform
        if self.processType==ProcessType.DETECTION:
            response = self.textract.start_document_text_detection(DocumentLocation={'S3Object': {'Bucket': self.bucket, 'Name': self.document}},
            NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

            print('Processing type: Detection')
            validType=True

        if self.processType==ProcessType.ANALYSIS:
            response = self.textract.start_document_analysis(DocumentLocation={'S3Object': {'Bucket': self.bucket, 'Name': self.document}},
                FeatureTypes=["TABLES", "FORMS"],
                NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn':self.snsTopicArn})
            
            print('Processing type: Analysis')
            validType=True
        
        if validType==False:
            print("Invalid processing type. Choose Detection or Analysis.")
            return
        print('Start Job Id: ' + response['JobId'])
        dotLine=0
        while jobFound == False:
            sqsResponse = self.sqs.receive_message(QueueUrl=self.sqsQueueUrl,
MessageAttributeNames=['ALL'],
MaxNumberOfMessages=10)


f_name = 'agapidou_2014'

#have to set up an aws account s3 bucket... (cloud object storage)
#for s3object, you need to specify bucket, name and version:
s3_obj =  {"Bucket": 'bpdcnpdfbucket', "Name": 'agapidou_2014.pdf'}  #creating dictionary obj

detect = client.start_document_text_detection(
    DocumentLocation={'S3Object': s3_obj},
    NotificationChannel={
        'SNSTopicArn': 'arn:aws:sns:us-west-1:760515291717:nlp_topic',
        'RoleArn': 'arn:aws:iam::760515291717:role/TextractRole'
    },
    OutputConfig={
        'S3Bucket': 'bpdcnpdfbucket',
        'S3Prefix': f_name
    })  #for detecting and analyzing text in multipage docs (asynchronous op), I would try both syncrhonous and asynchronous, maybe the synchronous works for pdfs with just a couple of pages?

print('detect job id: \n', detect['JobId']) 
detect_jobid = detect['JobId']


detect: 
 672a3182713fcdbb0a31b060eb98c50ec5096a2140e06662f2a364e235b3b008


In [2]:
doc_text = client.get_document_text_detection(
    JobId = detect_jobid)

In [7]:
print(doc_text)

{'JobStatus': 'IN_PROGRESS', 'DetectDocumentTextModelVersion': '1.0', 'ResponseMetadata': {'RequestId': '249cc7f1-d9a5-407c-896a-3044749ee4d2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '249cc7f1-d9a5-407c-896a-3044749ee4d2', 'content-type': 'application/x-amz-json-1.1', 'content-length': '66', 'date': 'Tue, 27 Apr 2021 22:57:06 GMT'}, 'RetryAttempts': 0}}
